# Digital sound synthesis and modulation

The last third of our signals and systems course typically covers: modulation and communication systems, Laplace and $z$-transforms, and feedback systems. In this lab, you can see how these all fit together in the  context of music and digital sound synthesis. 

Learning outcomes:
 - apply effects to digital sound using feedback, delay, and modulation
 - simulate the sound of plucked strings using the Karplus-Strong algorithm 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from IPython.display import Audio, Image
from IPython.core.display import HTML

## Part 1: musical effects and modulation

Modulation is defined in Oppenheim's 1996 Signals and Systems textbook as "the process of embedding an information-bearing signal into a second signal". In this course we explore two types of modulation: sinusoidal amplitude modulation, and frequency modulation.

Let's get started with some simple amplitude modulation. Below are some basic signals that can synthesize musical sounds (pro tip: SciPy's [signal processing module](https://docs.scipy.org/doc/scipy/reference/signal.html#waveforms) (documentation opens in new tab) contains some built-in definitions of simple waveforms as well!).

In [3]:
def cos_wave(t, freq, phase=0):
    """Cosine wave generator."""
    return np.cos(2 * np.pi * freq * t + phase)

def square_wave(t, freq):
    """Square wave generator."""
    return 4 * np.floor(freq * t) - 2 * np.floor(2 * freq * t)

def sawtooth_wave(t, freq):
    """Sawtooth wave generator."""
    return 2 * (freq * t - np.floor(freq * t + 0.5))

def triangle_wave(t, freq):
    """Triangle wave generator."""
    return 2 * np.abs(freq * t - np.floor(freq * t + 0.5))

In [4]:
sample_rate = 48000 # Samples per second
length = 2          # Signal duration in seconds
frequency = 440     # In Hz

t_range = np.linspace(0, length, sample_rate * length)

square_440 = square_wave(t_range, frequency)

Have a listen:

In [5]:
Audio(square_440, rate=sample_rate)

**Question 1.1.** Amplitude modulation uses a modulating signal to change the *amplitude* of a carrier signal. Complete the function below to implement sinusoidal amplitude modulation. The function should accept a modulating signal $x(t)$ and a carrier frequency $\omega_c$, and return

$$
y(t) = x(t) \cos(\omega_c t)
$$

Note that you can use the `cos_wave` function defined above to simplify your implementation.

In [ ]:
def amplitude_modulation(signal, time_range, carrier_frequency):
    """Apply sinusoidal amplitude modulation.
    
    Args:
        signal (array[float]): The modulating signal.
        time_range (array[float]): The explicit times over which the signal has
            been sampled (in seconds).
        carrier_frequency (int): The frequency (in Hz) of the carrier signal.
            
    Returns: 
        array[float]: The modulated signal.
    """
    return

**Question 1.2.** Once your function is ready, try modulating a signal (e.g., the 440 Hz square wave) in two different ways:
 - with a very low carrier frequency (e.g., < 3 Hz) 
 - with a low frequency (e.g., ~10 Hz)
 - with a higher frequency (e.g., > 100 Hz)
 
How does the sound of the signal change?

This kind of low-frequency amplitude modulation, when done on a musical instrument, is called *tremolo* (see, for example, [this example on the violin](https://www.youtube.com/watch?v=uQPkjuozGDM) (video opens in new tab)). Many other effects rely not on manipulating the amplitude of the carrier, but rather its frequency and phase. This can take one of two forms: angle (phase) modulation, or frequency modulation. 

**Question 1.3.** An effect similar to tremolo is *vibrato*. If you play a stringed instrument, this is what happens when you move your finger up and down in the same plane as the fret board (like [this example on a guitar](https://youtu.be/E3Pkza4SFCw?t=41) (video opens in new tab)). Essentially, the frequency of the note is changing in an oscillatory manner.  Vibrato can be modeled using phase modulation in the following way:

$$
y(t) = \cos(\omega_c t + x(t))
$$

Implement this in the function below. Then, apply it to a carrier sinusoid with a frequency of $f_c = 440 $ Hz and consider the following:
 - What shape of $x(t)$ should you use here to mimic the effect in the guitar video? 
 - How should it be modified to control the strength of modulation that is applied? (On a guitar, this corresponds to how much you bend the strings)

In [ ]:
def phase_modulation(signal, time_range, carrier_frequency):
    """Apply phase modulation to a carrier signal.
    
    Args:
        signal (array[float]): The signal (x(t)) that we modulate with.
        time_range (array[float]): The explicit times over which the signal has
            been sampled (in seconds).
        carrier_frequency (int): The frequency (in Hz) of the cos wave that
            we modulate.
            
    Returns: 
        array[float]: The modulated signal.
    """
    return

Let's try one more. If you are a guitarist and have ever played with effect pedals, you may have encountered settings called "phaser" or "flanger". These work by splitting a signal into two, putting one part through set of allpass filters that affect the phase in various ways, and then recombining the lines.

**Question 1.4.** A flanger (give it a listen [in this video](https://youtu.be/NAqQvs_WXs8?t=41) (opens in new tab)) is a delay-based effect described by the following block diagram:

<img src="flanger.png" width=700> 

The amount of delay is varied periodically using a low-frequency oscillator (LFO). Complete the implementation of the flanger below. I've set up a LFO already using a triangle wave, to evaluate the delay amounts. Your job is to construct the output signal based on the block diagram.

In [ ]:
def flanger(signal, lfo_delay=0.002, lfo_freq=0.25, sample_rate=48000):
    """Apply flanging to a signal. 
    
    Args:
        signal (array[float]): The signal we would like to flange.
        lfo_delay (float): A delay time (in ms). 
        lfo_freq (float): The frequency (in Hz) of the low-frequency oscillator
            used to determine the delay time.
        sample_rate (int): The sample rate of the signal (in Hz).
            
    Returns: 
        array[float]: The flangelated (?) signal.
    """
    # Generate a slow-varying triangle triangle wave
    low_frequency_oscillator = triangle_wave(np.arange(len(signal)) / sample_rate, lfo_freq)
    
    # Compute the number of samples we must delay by from the delay time, 
    # sample rate, and low-frequency oscillator
    delay_amount = np.around(lfo_delay * sample_rate * low_frequency_oscillator)
    
    # Clip the delay amount so it never tries to look outside the signal bounds
    delay_amount[delay_amount < 0] = 0
    delay_amount[delay_amount > len(signal) - 1] = len(signal) - 1
    
    #######################
    #   YOUR CODE HERE    #
    #######################
    
    # Use the signal and delay amounts to compute the output signal
    
    return

**Question 1.5.** Apply the flanger effect to the 440 Hz square wave signal. Then, try varying the frequency of the LFO triangle wave, as well as the delay time. What happens to the sound when you increase them? Why do you think that is?

If you're interested in seeing some more effects, [this Colab notebook](https://colab.research.google.com/github/gened1080/guitar-effects/blob/master/guitar_effects.ipynb#scrollTo=cKvnkb44y3cQ) (opens in new tab) gives a good overview and includes simple explanations of the signal processing involved.

## Part 2: the Karplus-Strong algorithm

While the above signals can be used to create some pretty cool digital music, they don't really have a instrument-quality sound to them. Back in the 80s, a different type of algorithm was designed in order to digitally replicate the sound of plucked strings. It is know as the "digitar" (for digital guitar), or the Karplus-Strong algorithm. It is shown in following block diagram:

<img src="ks-schematic.png" width=500>

(*By PoroCYon - Own work, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=44825728*)

There are a number of "Signals and Systems" components present here:
 - delay
 - feedback
 - a lowpass filter
 
Loosely, the initial burst corresponds to a pluck, and the filtered, delayed portion simulates the decay of the sound over time. The length of the initial burst is related to the frequency of the sound, as we will explore below.

**Question 2.1.** First things first: implement a function that will produce a burst of white noise, i.e., a random array with values between -1.0 and 1.0. *Hint: check out the [NumPy documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) (opens in new tab) for helpful functions that will assist you with generating random numbers.*


In [ ]:
def noise_burst(N_samples, waveform_type="bernoulli"):
    """Create a burst of white noise to initialize the Karplus-Strong algorithm.
              
    Use randomly-selected values between -1.0 and 1.0 according to the 
    following distributions:
      - Bernoulli w/p=0.5 (i.e., 1.0 or -1.0 with equal probability)
      - Uniform
      - Gaussian with mean 0
      
    Note that the data type of the random values must be a float, so be 
    sure to cast appropriately.
            
    Args:
        N_samples (int): The length of the sample.
        waveform_type (str): The type of random noise. Can be "bernoulli",
            "uniform", or "gaussian"
            
    Returns: 
        array[float]: The noise burst.            
    """

    return

Try out your burst generator; use a fairly high number of samples (e.g., 48000 will give a 1s sample) in order to generate a sample that is long enough to listen to.

Equipped with our white noise, let's get into the details of the algorithm.


<img src="ks-schematic.png" width=500>


(*By PoroCYon - Own work, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=44825728*)


First, the noise burst of $L$ samples is taken. These $L$ samples are sent to the output, as well as into a feedback loop that is first delayed by $L$ samples. In the feedback loop, the samples are sent through a DT lowpass filter with gain $g$ < 1, according to 

$$
y[n] = 0.5 g (x[n] + x[n-1])
$$

The filtered samples are sent to the output, but another copy of them is fed back into the feedback loop to be delayed and filtered again.

**Question 2.2.** Why do you think the gain of the filter is < 1? Think back to the beginning of the course; what important system property is this related to?

**Question 2.3.** Using the block diagram and description above, implement the Karplus-Strong synthesis algorithm using the starter code.

*Hint: a convenient way of implementing the algorithm is to make use of a [ring buffer](https://en.wikipedia.org/wiki/Circular_buffer) (article opens in new tab) for the delay/feedback aspect.*

In [ ]:
def karplus_strong(burst, num_feedback_loops=20, gain=0.996):
    """Implement the Karplus-Strong synthesis algorithm.
              
    Args:
        burst (array[float]): The initial noise burst.
        num_feedback_loops (int): The number of times to go through the feedback line.
        gain (float): The gain applied to the lowpass filter.
            
    Returns: 
        array[float]: The synthesized signal. It should have length 
        len(burst) * num_feedback_loops.
    """

    return 

**Question 2.4.** Once you're ready, try out the algorithm with a Bernoulli noise burst with 800 samples. Listen to the audio, and also plot the signal values as a function of time. Does it sound like a guitar?

**Question 2.5.** Based on how the algorithm works, what is the relationship between the sample length, sample rate, and output sound frequency? Implement this as a function below.

In [ ]:
def burst_length_from_frequency(frequency, sample_rate=48000):
    """Compute the length of noise burst needed for the KS algorithm
    to generate a tone of a given frequency.
              
    Args:
        frequency (float): A frequency in Hz.
        sample_rate (int): The sampling rate (samples / second).
            
    Returns: 
        int: The length of noise burst to create.
    """
    return

**Question 2.6.** Plot the output as a function of time for a couple different frequencies (in Hz). Use the helper function below and adjust if needed. How does the frequency of the note affect the rate of decay?

In [ ]:
def pluck_string(frequency):
    """Runs the KS algorithm to produce a sound at the provided frequency."""
    burst_length = burst_length_from_frequency(frequency)
    burst = noise_burst(burst_length, "bernoulli")
    return karplus_strong(burst, num_feedback_loops=50)

**Question 2.7 (optional)**. Implement a modified KS algorithm that includes a stretch factor that can be used to adjust the duration of the notes. The following resources may be helpful for figuring out how to do this:
 - https://flothesof.github.io/Karplus-Strong-algorithm-Python.html 
 - https://theory.stanford.edu/~blynn/sound/karplusstrong.html

**Question 2.8.** A variant of the Karplus-Strong algorithm involves starting with a "noise burst" of all 1s, and then randomly changing the sign of the filter output like so:

$$
y[n] = \begin{cases}
 0.5 g (x[n] + x[n-1]), & \hbox{w/probability } b \\
 -0.5 g (x[n] + x[n-1]), & \hbox{w/probability } 1- b\\
\end{cases}
$$

Implement this modification in the code block below, and try it out for a couple different values of $b$. What instrument do you think this could be used to simulate?

In [ ]:
def karplus_strong_modified(burst, num_feedback_loops=20, gain=0.996, b=0.5):
    """Implement the modified Karplus-Strong synthesis algorithm.
              
    Args:
        burst (array[float]): The initial noise burst. For the modified algorithm
            it should be all 1s.
        num_feedback_loops (int): The number of times to go through the feedback line.
        gain (float): The gain applied to the lowpass filter.
        b (float): The probability with which the sign of the filter changes.
            
    Returns: 
        array[float]: The synthesized signal. It should have length 
        len(burst) * num_feedback_loops.
    """

    return

**Question 2.9 (optional).** Using the tools above, compose a short piece of music! It can be totally new, or something you already know. Feel free to set up any additional software structures you need that will help you, e.g., [mappings between piano keys to frequencies](https://assets.becauselearning.com/images/files/000/001/001/original/blob?1472189156) (image opens in new tab), a chord generator, or a data structure to represent notes and their durations in order specify the notes in a song. 